## This work book is for using the AWS crimedbmysql datababase


This is the raw connection information
"""
host="crimedbmysql.cspoouh9lugd.us-east-2.rds.amazonaws.com"
user="crimeadmin"
password="4DAnuuSBWVQ92w!F"
database="crimedb_mysql"

"""
#### SM Example usage to connect to DB with a cursor
#Connect to db
db_connection, cursor = connect_to_database()
print(f"Connected to MySQL database: {db_connection}")

In [1]:
!pip install mysql-connector-python requests

In [2]:
import mysql.connector
import csv 


In [3]:
#SM Define some standard functions

#SM Function to connect to the Database
def connect_to_database():
    # Crime Analysis AWS MySQL server details
    db_config = {
        'host': 'crimedbmysql.cspoouh9lugd.us-east-2.rds.amazonaws.com',
        'user': 'crimeadmin',
        'password': '4DAnuuSBWVQ92w!F',
        'database': 'crimedb_mysql',
    }

    try:
        # Connect to the MySQL server
        db_connection = mysql.connector.connect(**db_config)

        # Create a cursor object to execute SQL queries
        cursor = db_connection.cursor()

        print("Connected to the database.")
        return db_connection, cursor

    except mysql.connector.Error as err:
        print(f"Error: {err}")
        return None, None

def close_connection(db_connection, cursor):
    # Close the cursor and connection
    if cursor:
        cursor.close()
    if db_connection:
        db_connection.close()
        print("Connection closed.")

def delete_table(cursor, table_name):
    # Delete the table
    delete_table_query = f"DROP TABLE IF EXISTS {table_name}"
    cursor.execute(delete_table_query)
    print(f"Table '{table_name}' deleted successfully.")
    



In [4]:
# SM attempting to create table creation query for train data
create_table_train_data = """
CREATE TABLE Clean_Train_Data (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    Year INT,
    Month DATE,
    Station_ID INT,
    Station_Name VARCHAR(255),
    Avg_Weekday_Rides FLOAT,
    Avg_Saturday_Rides FLOAT,
    Avg_Sunday_Holiday_Rides FLOAT,
    Monthly_Total INT,
    Lat_Lon POINT,
    Community VARCHAR(255),
    Comm_Num INT,
    SHAPE_AREA DOUBLE,
    SHAPE_LEN DOUBLE
);
"""

# Connect to the MySQL database
try:
    db_connection, cursor = connect_to_database()
    cursor = db_connection.cursor()

    # Execute the table creation query
    cursor.execute(create_table_train_data)
    print("Table created successfully!")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    # Close the database connection
    if db_connection.is_connected():
        cursor.close()
        db_connection.close()
        print("Connection closed.")

Connected to the database.
Error: 1050 (42S01): Table 'Clean_Train_Data' already exists
Connection closed.


In [5]:
#SM attempting to Insert Clean_Train_DATA data into table named Clean_Train_Data
#SM this hangs when running this cell 
# Specify the path to your CSV file
csv_file_path = './data/Cleaned/CleanedDatasets/Train_Data.csv'

# SQL query to insert data into the table
insert_query = """
INSERT INTO Clean_Train_Data
(Year, Month, Station_ID, Station_Name, Avg_Weekday_Rides, Avg_Saturday_Rides, Avg_Sunday_Holiday_Rides, Monthly_Total, Lat_Lon, Community, Comm_Num, SHAPE_AREA, SHAPE_LEN)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, ST_GeomFromText(%s), %s, %s, %s, %s)
"""

# Connect to the MySQL database
try:
    #Connect to db
    db_connection, cursor = connect_to_database()
    cursor = db_connection.cursor()

    # Open and read data from the CSV file
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # Skip header row

        # Execute the insert query for each row of data
        for row in csv_reader:
            cursor.execute(insert_query, tuple(row))

    # Commit the changes
    db_connection.commit()
    print("Data inserted successfully!")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    # Close the database connection
    if db_connection.is_connected():
        cursor.close()
        db_connection.close()
        print("Connection closed.")

Connected to the database.
Connection closed.


KeyboardInterrupt: 

In [ ]:
#SM show all table names by connecting to db and fetching all table names and then close the conneciton
# Check if the connection is successful and print db object name
#Connect to db
db_connection, cursor = connect_to_database()
print(f"Connected to MySQL database: {db_connection}")

if db_connection.is_connected():
    print(f"Connected to MySQL database: {db_connection}")

    # Create a cursor
    cursor = db_connection.cursor()

    # Get the list of tables in the database
    cursor.execute("SHOW TABLES")

    # Fetch and print the results
    tables = cursor.fetchall()
    print("\nTables in the database:")
    for table in tables:
        print(table[0])

    # Close the cursor and connection
    cursor.close()
    db_connection.close()
    print("closed cursor and connection to db")
else:
    print("Connection failed")

In [ ]:
#SM connect to db and fetch and print all Tables, columns, and rows

#Connect to db
db_connection, cursor = connect_to_database()

# Check if the connection is successful
if db_connection.is_connected():
    print(f"Connected to MySQL database: {db_connection}")

    # Create a cursor
    cursor = db_connection.cursor()

    # Get the list of tables in the database
    cursor.execute("SHOW TABLES")

    # Fetch and print the results
    tables = cursor.fetchall()
    print("\nTables in the database:")
    for table in tables:
        table_name = table[0]
        print(f"\nData in table '{table_name}':")

        # Fetch all rows from the table
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        # Fetch column names
        cursor.execute(f"DESC {table_name}")
        columns = [column[0] for column in cursor.fetchall()]

        # Print column names
        print("\t".join(columns))

        # Print data
        for row in rows:
            print("\t".join(str(value) for value in row))

    # Close the cursor and connection
    cursor.close()
    db_connection.close()
else:
    print("Connection failed")

#### Code Snippets below

#SM example code for simple functions, but not written to allow for different data

def create_table(cursor, table_name):
    # Create a sample table
    create_table_query = f"""CREATE TABLE IF NOT EXISTS {table_name} (
        id INT AUTO_INCREMENT PRIMARY KEY,
        crime_type VARCHAR(255) NOT NULL,
        location VARCHAR(255) NOT NULL,
        date_reported DATE
    )
    """

    cursor.execute(create_table_query)
    print(f"Table '{table_name}' created successfully.")
    
    """
def insert_data(cursor, table_name, data):
    # Insert data into the table
    insert_query = f"INSERT INTO {table_name} (crime_type, location, date_reported) VALUES (%s, %s, %s)"
    cursor.execute(insert_query, data)
    print("Data inserted successfully.")

def update_data(cursor, table_name, data):
    # Update data in the table
    update_query = f"UPDATE {table_name} SET crime_type = %s WHERE location = %s"
    cursor.execute(update_query, data)
    print(f"Data updated successfully in '{table_name}'.")

def delete_data(cursor, table_name, data):
    # Delete data from the table
    delete_query = f"DELETE FROM {table_name} WHERE location = %s"
    cursor.execute(delete_query, data)
    print(f"Data deleted successfully from '{table_name}'.")

def select_data(cursor, table_name):
    # Select data from the table
    select_query = f"SELECT * FROM {table_name}"
    cursor.execute(select_query)
    print(f"Selected data from '{table_name}'.")

In [ ]:
#Connect to db
db_connection, cursor = connect_to_database()

# Check if the connection is successful
if db_connection.is_connected():
    print(f"Connected to MySQL database: {db_connection}")

    # Create a cursor
    cursor = db_connection.cursor()

    # Close the cursor and connection
    cursor.close()
    db_connection.close()
else:
    print("Connection failed")

In [ ]:
#SM for Train data

import csv
import mysql.connector


# Define the table creation query
create_table_query = """
CREATE TABLE your_table_name (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    Year INT,
    Month DATE,
    Station_ID INT,
    Station_Name VARCHAR(255),
    Avg_Weekday_Rides FLOAT,
    Avg_Saturday_Rides FLOAT,
    Avg_Sunday_Holiday_Rides FLOAT,
    Monthly_Total INT,
    Lat_Lon POINT,
    Community VARCHAR(255),
    Comm_Num INT,
    SHAPE_AREA DOUBLE,
    SHAPE_LEN DOUBLE
);
"""


# Specify the path to your CSV file
csv_file_path = '/data/Cleaned/CleanedDatasets/Train_Data.csv'

# SQL query to insert data into the table
insert_query = """
INSERT INTO your_table_name 
(Year, Month, Station_ID, Station_Name, Avg_Weekday_Rides, Avg_Saturday_Rides, Avg_Sunday_Holiday_Rides, Monthly_Total, Lat_Lon, Community, Comm_Num, SHAPE_AREA, SHAPE_LEN)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

# Connect to the MySQL database
try:
    #Connect to db
    db_connection, cursor = connect_to_database()
    cursor = db_connection.cursor()

    # Open and read data from the CSV file
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.reader(csv_file)
        next(csv_reader)  # Skip header row

        # Execute the insert query for each row of data
        for row in csv_reader:
            cursor.execute(insert_query, tuple(row))

    # Commit the changes
    connection.commit()
    print("Data inserted successfully!")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    # Close the database connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Connection closed.")

# Define the table creation query
create_table_train_data = """
CREATE TABLE Clean_Train_Data (
    ID INT AUTO_INCREMENT PRIMARY KEY,
    Year INT,
    Month DATE,
    Station_ID INT,
    Station_Name VARCHAR(255),
    Avg_Weekday_Rides FLOAT,
    Avg_Saturday_Rides FLOAT,
    Avg_Sunday_Holiday_Rides FLOAT,
    Monthly_Total INT,
    Lat_Lon POINT,
    Community VARCHAR(255),
    Comm_Num INT,
    SHAPE_AREA DOUBLE,
    SHAPE_LEN DOUBLE
);
"""

# Connect to the MySQL database
try:
    db_connection, cursor = connect_to_database()
    cursor = connection.cursor()

    # Execute the table creation query
    cursor.execute(create_table_train_data)
    print("Table created successfully!")

except mysql.connector.Error as err:
    print(f"Error: {err}")

finally:
    # Close the database connection
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("Connection closed.")

####TESING CREATE TABLE CODE

CREATE TABLE transit_data (
    Year INT,
    Month DATE,
    Station_ID INT,
    Station_Name VARCHAR(255),
    Avg_Weekday_Rides DECIMAL(10, 2),
    Avg_Saturday_Rides DECIMAL(10, 2),
    Avg_Sunday_Holiday_Rides DECIMAL(10, 2),
    Monthly_Total INT,
    Lat_Lon POINT,
    Community VARCHAR(255),
    Comm_Num INT,
    SHAPE_AREA DECIMAL(18, 2),
    SHAPE_LEN DECIMAL(18, 5)
);

-- Load data from CSV file into the table
LOAD DATA INFILE '/path/to/your/file.csv'
INTO TABLE transit_data
FIELDS TERMINATED BY ',' ENCLOSED BY '"'
LINES TERMINATED BY '\r\n'
IGNORE 1 LINES; -- Ignore the header row

#### SM Example code to import data into the
#SM getting error on types compare to dataset 
#SM enter data from Crimes_-_2001_to_Present_20231108 csv file
# Check if the connection is successful
if connection.is_connected():
    print(f"Connected to MySQL database: {database}")

    # Create a cursor
    cursor = connection.cursor()

    # Create a new table (if not exists)
    create_table_query = """
    CREATE TABLE IF NOT EXISTS CrimeData (
        ID INT PRIMARY KEY,
        CaseNumber VARCHAR(255),
        Date DATE,
        Block VARCHAR(255),
        IUCR VARCHAR(10),
        PrimaryType VARCHAR(255),
        Description VARCHAR(255),
        LocationDescription VARCHAR(255),
        Arrest BOOLEAN,
        Domestic BOOLEAN,
        Beat INT,
        District INT,
        Ward INT,
        CommunityArea INT,
        FBICode VARCHAR(5),
        XCoordinate FLOAT,
        YCoordinate FLOAT,
        Year INT,
        UpdatedOn TIMESTAMP,
        Latitude FLOAT,
        Longitude FLOAT,
        Location VARCHAR(255),
        HistoricalWards20032015 INT,
        ZipCodes INT,
        CommunityAreas INT,
        CensusTracts INT,
        Wards INT,
        BoundariesZIPCodes INT,
        PoliceDistricts INT,
        PoliceBeats INT
    )
    """
    cursor.execute(create_table_query)
    print("CrimeData table created successfully.")

    # Read data from the CSV file and insert into the 'CrimeData' table
    with open(csv_file_path, 'r') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        # Skip the header row
        next(csv_reader, None)

        insert_query = """
        INSERT INTO CrimeData (
            ID, CaseNumber, Date, Block, IUCR, PrimaryType, Description,
            LocationDescription, Arrest, Domestic, Beat, District, Ward,
            CommunityArea, FBICode, XCoordinate, YCoordinate, Year, UpdatedOn,
            Latitude, Longitude, Location, HistoricalWards20032015, ZipCodes,
            CommunityAreas, CensusTracts, Wards, BoundariesZIPCodes,
            PoliceDistricts, PoliceBeats
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        for row in csv_reader:
            data_to_insert = (
                int(row['ID']),
                row['Case Number'],
                row['Date'],
                row['Block'],
                row['IUCR'],
                row['Primary Type'],
                row['Description'],
                row['Location Description'],
                row['Arrest'].lower() == 'true',  # Assuming 'Arrest' is 'true' or 'false' in the CSV
                row['Domestic'].lower() == 'true',
                int(row['Beat']),
                int(row['District']),
                int(row['Ward']),
                int(row['Community Area']),
                row['FBI Code'],
                float(row['X Coordinate']),
                float(row['Y Coordinate']),
                int(row['Year']),
                row['Updated On'],
                float(row['Latitude']),
                float(row['Longitude']),
                row['Location'],
                int(row['Historical Wards 2003-2015']),
                int(row['Zip Codes']),
                int(row['Community Areas']),
                int(row['Census Tracts']),
                int(row['Wards']),
                int(row['Boundaries - ZIP Codes']),
                int(row['Police Districts']),
                int(row['Police Beats'])
            )
            cursor.execute(insert_query, data_to_insert)

    connection.commit()
    print("Data inserted into the CrimeData table.")

    # Close the cursor and connection
    cursor.close()
    connection.close()
else:
    print("Connection failed")

In [ ]:
#SM CREATE TABLE CODE

CREATE TABLE clean_train_data (
    Year INT,
    Month DATE,
    Station_ID INT,
    Station_Name VARCHAR(255),
    Avg_Weekday_Rides DECIMAL(10, 2),
    Avg_Saturday_Rides DECIMAL(10, 2),
    Avg_Sunday_Holiday_Rides DECIMAL(10, 2),
    Monthly_Total INT,
    Lat_Lon POINT,
    Community VARCHAR(255),
    Comm_Num INT,
    SHAPE_AREA DECIMAL(18, 2),
    SHAPE_LEN DECIMAL(18, 5)
);

-- Load data from CSV file into the table
LOAD DATA INFILE '/data/Cleaned/CleanedDatasets/Train_Data.csv'
INTO TABLE transit_data
FIELDS TERMINATED BY ',' ENCLOSED BY '"'
LINES TERMINATED BY '\r\n'
IGNORE 1 LINES; -- Ignore the header row

In [ ]:

# Replace 'your_table_name' with the actual name of the table you want to delete
table_to_delete = 'Clean_Train_Data'

#Connect to db
db_connection, cursor = connect_to_database()
print(f"Connected to MySQL database: {db_connection}")

if db_connection.is_connected():
    print(f"Connected to MySQL database: {db_connection}")

    # Create a cursor
    cursor = db_connection.cursor()

    # Get the list of tables in the database
    cursor.execute("SHOW TABLES")

    # Fetch and print the results
    delete_table(cursor, {table_to_delete})

    # Close the cursor and connection
    cursor.close()
    db_connection.close()
    print("closed cursor and connection to db")
else:
    print("Connection failed")

In [ ]:
#Connect to db
db_connection, cursor = connect_to_database()
print(f"Connected to MySQL database: {db_connection}")

if db_connection.is_connected():
    print(f"Connected to MySQL database: {db_connection}")

    # Create a cursor
    cursor = db_connection.cursor()

    # Get the list of tables in the database
    cursor.execute("SHOW TABLES")

    # Fetch and print the results
    tables = cursor.fetchall()
    print("\nTables in the database:")
    for table in tables:
        print(table[0])

    # Fetch and print the results
    tables = cursor.fetchall()
    print("\nTables in the database:")
    for table in tables:
        table_name = table[0]
        print(f"\nData in table '{table_name}':")

        # Fetch all rows from the table
        cursor.execute(f"SELECT * FROM {table_name}")
        rows = cursor.fetchall()

        # Fetch column names
        cursor.execute(f"DESC {table_name}")
        columns = [column[0] for column in cursor.fetchall()]

        # Print column names
        print("\t".join(columns))

        # Print data
        for row in rows:
            print("\t".join(str(value) for value in row))

    # Close the cursor and connection
    cursor.close()
    db_connection.close()
    print("closed cursor and connection to db")
else:
    print("Connection failed")